<a href="https://colab.research.google.com/github/AmiriChaima/Argument-detection/blob/main/Prameter_optimization_Bert_Claim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installations

In [ ]:
! pip install -q transformers
! pip install -q datasets
! pip install -q scikit-learn
! pip install -q nltk
! pip install -q imbalanced-learn
! pip install -q optuna
! pip install -q transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
import transformers
print(transformers.__version__)

import datasets
print(datasets.__version__)

import sklearn
print(sklearn.__version__)

import nltk
print(nltk.__version__)

import imblearn
print(imblearn.__version__)

import optuna
print(optuna.__version__)

4.33.3
2.14.5
1.2.2
3.8.1
0.10.1
3.3.0


In [ ]:
# set model as required (a whole list of possible pre-trained Transformer models here: https://huggingface.co/transformers/pretrained_models.html)
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [ ]:
# required to use the inbuilt Transformers Trainer class
from datasets import load_dataset, load_metric

In [ ]:
# set metric as required. Available inbuilt metrics (extend the datasets Metric class) are here: https://github.com/huggingface/datasets/tree/master/metrics. Can write your own custom metric too.
metric = load_metric('matthews_correlation')

# uncomment the below to print selected metric details
# print(metric)

<ipython-input-5-ca3ff90b5a85>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('matthews_correlation')


### Get data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# read CSVs
url = 'https://raw.githubusercontent.com/AmiriChaima/MyData/main/data/Premise_Data.csv'


# read as pandas DataFrame (to be converted to dataset later)
df= pd.read_csv(url, skip_blank_lines=True, encoding='ISO-8859-1', delimiter=";")
df


,ID,Num_Doc,Statement,Type
0,305:11,Atom_902,the bottom left corner of the tabs are missing...,Premise: Specific usage experience
1,305:15,Atom_902,"Tabs were redesigned and shipped,",Premise: Evidence that the issue is or can be ...
2,306:8,Atom_995,Maybe dupes on labels is better with a `consol...,Premise: Evidence that the issue is or can be ...
3,307:14,Atom_1159,whether it's an inability to manage panes when...,Premise: Specific usage experience
4,308:7,Atom_1708,![screen shot 2014-03-05 at 10 11 02 pm](https...,Premise: Visual content or supporting file
...,...,...,...,...
793,607:5,VsCode_117499,https://github.com/microsoft/vscode/blob/7222d...,Premise: Visual content or supporting file
794,607:9,VsCode_117499,![TreeviewGlyphs](https://user-images.githubus...,Premise: Visual content or supporting file
795,607:10,VsCode_117499,"For example, the first entry in this treeview ...",Premise: Specific usage experience
796,608:7,VsCode_122839,"""Search include pattern"" and ""Search exclude p...",Premise: Clarification/ providing additional i...


In [ ]:
possible_labels = df.Type.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict
df['Type'] = df.Type.replace(label_dict)


In [ ]:
print("\n\nOriginal dataframe\n")
df=df.drop(columns=['ID','Num_Doc'],axis=1)
df




Original dataframe



,Statement,Type
0,the bottom left corner of the tabs are missing...,0
1,"Tabs were redesigned and shipped,",1
2,Maybe dupes on labels is better with a `consol...,1
3,whether it's an inability to manage panes when...,0
4,![screen shot 2014-03-05 at 10 11 02 pm](https...,2
...,...,...
793,https://github.com/microsoft/vscode/blob/7222d...,2
794,![TreeviewGlyphs](https://user-images.githubus...,2
795,"For example, the first entry in this treeview ...",0
796,"""Search include pattern"" and ""Search exclude p...",5


In [ ]:
df = df.rename(columns={'Type': 'label'})


In [ ]:
from collections import Counter
Counter(df["label"])

Counter({0: 178, 1: 128, 2: 300, 3: 81, 4: 43, 5: 68})

<br>Split (stratified along label) into train, eval, test sets. Ratio here: 70:15:15.

In [ ]:
train, temp = train_test_split(df, random_state=2018, test_size=0.3, stratify=df['label'])
eval, test = train_test_split(temp, random_state=2018, test_size=0.5, stratify=temp['label'])
train

,Statement,label
273,The header tag is just one case that triggers it.,5
145,![smc-linebreak-20170422](https://cloud.github...,2
474,I get\r\r<img,2
715,to **provide a text prompt** like notepad++ ra...,4
606,![screen shot 2017-04-26 at 4 17 49 pm](https:...,2
...,...,...
675,Here is another sample:,2
325,since it is not a user-facing item,5
539,See attached:\r\rNo log at Error level:\r![ima...,2
570,with commit,1


In [ ]:
train = train.reset_index(drop=True)
eval = eval.reset_index(drop=True)
test = test.reset_index(drop=True)

### EDA implementation

In [ ]:
# imports for EDA
import random
from random import shuffle
random.seed(1)

import re

import nltk
nltk.download("wordnet")
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#stop words list
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our',
			'ours', 'ourselves', 'you', 'your', 'yours',
			'yourself', 'yourselves', 'he', 'him', 'his',
			'himself', 'she', 'her', 'hers', 'herself',
			'it', 'its', 'itself', 'they', 'them', 'their',
			'theirs', 'themselves', 'what', 'which', 'who',
			'whom', 'this', 'that', 'these', 'those', 'am',
			'is', 'are', 'was', 'were', 'be', 'been', 'being',
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at',
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after',
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again',
			'further', 'then', 'once', 'here', 'there', 'when',
			'where', 'why', 'how', 'all', 'any', 'both', 'each',
			'few', 'more', 'most', 'other', 'some', 'such', 'no',
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
			'very', 's', 't', 'can', 'will', 'just', 'don',
			'should', 'now', '']

#cleaning up text
import re
import re

def get_only_chars(line):
    clean_line = ""

    # change this as required!
    # line = line.replace("’", "")
    # line = line.replace("'", "")
    line = line.replace("-", " ")  # replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")

    for char in line:
        # can set own range as required.
        if char in 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.?!.;:\'\" ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +', ' ', clean_line)  # delete extra spaces
    if len(clean_line) > 0 and clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################


def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word):
		for l in syn.lemmas():
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'])
			synonyms.add(synonym)
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################

# def random_deletion(words, p):

# 	#obviously, if there's only one word, don't delete it
# 	if len(words) == 1:
# 		return words

# 	#randomly delete words with probability p
# 	new_words = []
# 	for word in words:
# 		r = random.uniform(0, 1)
# 		if r > p:
# 			new_words.append(word)

# 	#if you end up deleting all words, just return a random word
# 	if len(new_words) == 0:
# 		rand_int = random.randint(0, len(words)-1)
# 		return [words[rand_int]]

# 	return new_words

def random_deletion(words, p):
    # Obviously, if there's only one word, don't delete it
    if len(words) == 1:
        return words

    # Randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    # If you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words) - 1)
        return [words[rand_int]]

    return new_words



########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

# def random_swap(words, n):
# 	new_words = words.copy()
# 	for _ in range(n):
# 		new_words = swap_word(new_words)
# 	return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        if not new_words or len(new_words) < 2:
            break  # Break the loop if new_words is empty or has less than 2 words
        new_words = swap_word(new_words)
    return new_words


def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################

# def random_insertion(words, n):
# 	new_words = words.copy()
# 	for _ in range(n):
# 		add_word(new_words)
# 	return new_words

def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        if not new_words:
            break  # Break the loop if new_words is empty
        add_word(new_words)
    return new_words

# def add_word(new_words):
# 	synonyms = []
# 	counter = 0
# 	while len(synonyms) < 1:
# 		random_word = new_words[random.randint(0, len(new_words)-1)]
# 		synonyms = get_synonyms(random_word)
# 		counter += 1
# 		if counter >= 10:
# 			return
# 	random_synonym = synonyms[0]
# 	random_idx = random.randint(0, len(new_words)-1)
# 	new_words.insert(random_idx, random_synonym)

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        if not new_words:
            return  # Return if new_words is empty
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)


########################################################################
# main data augmentation function
########################################################################

def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    sentence = get_only_chars(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word != '']
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug / 4) + 1

    # sr
    if (alpha_sr > 0):
        n_sr = max(1, int(alpha_sr * num_words))
        for _ in range(num_new_per_technique):
            a_words = synonym_replacement(words, n_sr)
            augmented_sentences.append(' '.join(a_words))

    # ri
    if (alpha_ri > 0):
        n_ri = max(1, int(alpha_ri * num_words))
        for _ in range(num_new_per_technique):
            a_words = random_insertion(words, n_ri)
            augmented_sentences.append(' '.join(a_words))

    # rs
    if (alpha_rs > 0):
        n_rs = max(1, int(alpha_rs * num_words))
        for _ in range(num_new_per_technique):
            a_words = random_swap(words, n_rs)
            augmented_sentences.append(' '.join(a_words))

    # rd
    if (p_rd > 0 and num_words > 0):  # Ensure that words is not empty
        for _ in range(num_new_per_technique):
            a_words = random_deletion(words, p_rd)
            augmented_sentences.append(' '.join(a_words))

    augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    # trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    # append the original sentence
    augmented_sentences.append(sentence)

    return augmented_sentences


In [ ]:
def gen_eda(orig_train_df, output_train_df, alpha_sr, alpha_ri, alpha_rs, alpha_rd, num_aug=9):

  """
  Parameters:
  orig_train_df: unaugmented train (pandas) DataFrame
  output_train_df: empty DataFrame
  alpha_sr = augmentation parameter for synonym replacement
  alpha_ri = augmentation parameter for random insertion
  alpha_rs = augmentation parameter for random swapping
  alpha_rd = augmentation parameter for random deletion
  num_aug = number of generated sentences per input sentence.
  (generates (int(num_aug/4) + 1) sentences per task, shuffles dataset and trims to num_aug generated per sentence.)

  Returns:
  output_train_df = pandas DataFrame containing sentences from the original DataFrame + newly generated sentences.
  Size of output_train_df = (size of orig_train_df)*(num_aug + 1)
  """
  for idx in orig_train_df.index:
    label = orig_train_df.loc[idx, 'label']
    sentence = orig_train_df.loc[idx, 'Statement']
    aug_sentences = eda(sentence, alpha_sr=alpha_sr, alpha_ri=alpha_ri, alpha_rs=alpha_rs, p_rd=alpha_rd, num_aug=num_aug)
    d = {}
    d["Statement"] = aug_sentences
    d["label"] = [label for _ in range(len(aug_sentences))]
    df = pd.DataFrame(d)
    output_train_df = pd.concat([output_train_df, df], ignore_index=True)

  print("Generated", num_aug, "sentences per input sentence with EDA")
  return output_train_df

In [ ]:
# empty DataFrame
output_train = pd.DataFrame(columns=["Statement", "label"])
output_train

,Statement,label


**Recommended EDA hyperparameters here:**<br>(configurations for grid search)

```
alpha_sr in {0.05, 0.2}
alpha_ri in {0.05, 0.1}
alpha_rs in {0.05}
alpha_rd in {0.05}
num_aug in {4, 8}
```



In [ ]:
# augmented train DataFrame
aug_train = gen_eda(train, output_train, 0.05, 0.05, 0.05, 0.05,8)

Generated 8 sentences per input sentence with EDA


In [ ]:
# print("\n\nAugmented train dataframe\n")
# print(aug_train.head())

In [ ]:
print("\n\n# of datapoints before augmentation:", len(train))
print("# of datapoints after augmentation:", len(aug_train))



# of datapoints before augmentation: 558
# of datapoints after augmentation: 5022


### Class balancing: random oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
# for random undersampling, uncomment the below:
# from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

In [ ]:
## OVERSAMPLING
"""### Class balancing: random oversampling"""

from imblearn.over_sampling import RandomOverSampler
# for random undersampling, uncomment the below:
# from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

train_text = aug_train["Statement"]
train_labels = aug_train["label"]
print("#\n\nLabel count before oversampling:", Counter(train_labels))

X_train_np, y_train_l = train_text.to_numpy(), train_labels.to_list()
X_train_np = X_train_np.reshape((-1, 1))

# from sklearn.utils.multiclass import type_of_target - need this!
# print(type_of_target(tr_list))

ros = RandomOverSampler(sampling_strategy="minority")
X_resampled, y_resampled = ros.fit_resample(X_train_np, y_train_l)

# for random undersampling, uncomment the below:
# rus = RandomUnderSampler(sampling_strategy="majority")
# X_resampled, y_resampled = rus.fit_resample(X_train_np, y_train_np)

shape = (len(y_resampled),)
X_resampled = X_resampled.reshape(shape)

train_text = X_resampled
train_labels = y_resampled

print("Label count after oversampling:", Counter(train_labels))
# print("Label count after undersampling:", Counter(train_labels))


#

Label count before oversampling: Counter({2: 1890, 0: 1116, 1: 801, 3: 513, 5: 432, 4: 270})
Label count after oversampling: Counter({2: 1890, 4: 1890, 0: 1116, 1: 801, 3: 513, 5: 432})


In [ ]:
train = pd.DataFrame({'Statement': train_text, 'label': train_labels})
print("\n\nTrain dataset after augmentation and oversampling\n")
print(train.head())



Train dataset after augmentation and oversampling

                                           Statement  label
0  The header tag is just one case that triggers it.      5
1  The header tag is just one case that triggers ...      5
2  The header tag is just one case that activate it.      5
3  The header tag is just one grammatical case th...      5
4  The header tag triggers just one case that is it.      5


In [ ]:
# convert to dataset for easier Transformer integration
from datasets import Dataset

train_ds = Dataset.from_pandas(aug_train)
eval_ds = Dataset.from_pandas(eval)
test_ds = Dataset.from_pandas(test)

In [ ]:
# train_ds[0]

### Preprocessing data (tokenization)


In [ ]:
print("\n\nMetric details")
print(metric)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
# uncomment below to test
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def tokenize_function(ds):
  return tokenizer(ds["Statement"], truncation=True)

# uncomment below to test
# tokenize_function(train_ds[:5])

In [ ]:
# tokenize datasets (preprocessing)
encoded_train_ds = train_ds.map(tokenize_function, batched=True)
encoded_eval_ds = eval_ds.map(tokenize_function, batched=True)
encoded_test_ds = test_ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/5022 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [ ]:
print("\n\nEncoded train dataset sample\n")
print(encoded_train_ds[0])
print("\n\n")



Encoded train dataset sample

{'Statement': 'The header tag is just one case that triggers it.', 'label': 5, 'input_ids': [101, 1996, 20346, 6415, 2003, 2074, 2028, 2553, 2008, 27099, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}





### Fine tuning model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 6
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric_name = "matthews_correlation"

In [ ]:
# define TrainingArguments: arguments/ hyperparameters used in training/eval loop. You can find a complete list here: https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments.
# Most of these can be optimized
args = TrainingArguments(
    "Argumentation",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

In [ ]:
# needed during the training/eval loop (Trainer class instance). If you pass in compute_metrics=None (the default), evaluation is done with prediction loss only (no metric optimization objective)
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
####### Uncomment the below to train/evaluate the model without hyperparameter optimization

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_ds,
    eval_dataset=encoded_eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

### Hyperparameter search

In [ ]:
# uncomment the below to create a Dataset that includes only 1/num_shards of this dataset. useful for running distributed training, as it allows each worker to read a unique subset.
# if we do have distributed training, should shard much earlier (pre-EDA etc.) to optimize work distribution.


train_ds_hyp = encoded_train_ds.shard(index=1, num_shards=25)
eval_ds_hyp = encoded_eval_ds.shard(index=1, num_shards=10)

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
# Trainer is a simple, feature-complete training and eval loop for PyTorch, optimized for Hugging Face Transformers. https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_train_ds,
    eval_dataset=encoded_eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


By default, optuna optimizes the metric over:


```
"learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
"num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
"seed": trial.suggest_int("seed", 1, 40),
"per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64])
```

Customize a search space function, choosing suitable hyperparameters from the TrainingArguments list.



In [ ]:
def hp_space_optuna(trial):
  return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 7),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.3, log=True),
        "warmup_steps": trial.suggest_int("warmup_steps", 0, 100)
    }


Launch an hyperparameter search using optuna. The optimized quantity is determined by compute_objective, which defaults to a function returning the evaluation loss when no metric is provided, the sum of all metrics otherwise. To use this, we must have a model_init() in the Trainer, as the model is reinitialized in each run.

By default, n_trials=100, direction="minimize".

We don't need to define our own compute_objective function. The metric we specified in the Trainer instance is used as the optimization quantity. If no metric is provided, evaluation loss is used. If multiple metrics are provided, it uses the sum of all metrics.

In [ ]:
best_run = trainer.hyperparameter_search(hp_space_optuna, n_trials=3, direction="maximize")
# best_run = trainer.hyperparameter_search(n_trials=3, direction="maximize")

In [ ]:
# print details of best run (prints best run ID, maximized metric, and a dictionary of optimized hyperparameters)
print(best_run)

BestRun(run_id='2', objective=0.4542376885830343, hyperparameters={'learning_rate': 1.0553243226085032e-06, 'num_train_epochs': 4, 'per_device_train_batch_size': 16, 'weight_decay': 0.05200968350527944, 'warmup_steps': 79}, run_summary=None)
